In [2]:
# importing libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re

In [3]:
df = pd.read_csv("../data/flipkart_com-ecommerce_sample.csv")

In [4]:
df['category'] = df['product_category_tree'].apply(lambda x: x.split(">>")[0].strip()[2:])

In [5]:
req_cols = ['uniq_id', 'product_name', 'category', 'description', 'retail_price', 'discounted_price', 'brand']

new_df = df[req_cols]

In [6]:
new_df.loc[0:, 'tags'] = new_df['product_name'] + " " + new_df['description']

/tmp/ipykernel_133694/2548074365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[0:, 'tags'] = new_df['product_name'] + " " + new_df['description']


In [7]:
new_df.drop([553, 17299], inplace=True)

/tmp/ipykernel_133694/2045436239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop([553, 17299], inplace=True)


In [8]:
def process_category(cate: str):
    cleaned_cate = re.sub(r'[^A-Za-z0-9\s]', '', cate)
    return cleaned_cate.lower()

In [9]:
new_df.loc[0:, 'category'] = new_df['category'].apply(lambda x: process_category(x))

In [10]:
def preprocess_sent(sent: str):
    specific_stop_words = ["numbr", "rs", "flipkart", "buy", "com", "free","day", "cash", "replacement", "guarantee", "genuine", "key", "feature", "delivery","products", "product", "shipping", "online", "india", "shop"]
    nltk_stopwords = set(stopwords.words('english'))

    all_stopwords = nltk_stopwords.union(specific_stop_words)

    sent_lst = sent.lower().split(" ")
    sent_lst = [word for word in sent_lst if word not in string.punctuation]
    sent_lst = [word for word in sent_lst if word not in all_stopwords]

    cleaned_sent = " ".join(sent_lst)
    cleaned_sent = cleaned_sent.replace(r'\s+', ' ').replace(r'^\s+ |\s+?$', '').replace(r'\d+(\.\d+)?', 'numbr')
    
    return cleaned_sent

In [11]:
new_df.loc[:, 'tags'] = new_df['tags'].apply(lambda x: preprocess_sent(x))

In [12]:
# vectorize the tags column
vectorizer = TfidfVectorizer(max_features=5000)

tokenizer = vectorizer.fit_transform(new_df['tags'])

In [13]:
tokenizer.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(19998, 5000))

In [14]:
similarity = cosine_similarity(tokenizer)

: 